In [1]:
import yaml
import os

with open('data.yaml', 'r') as file:
    data = yaml.safe_load(file)

print("Train path:", os.path.exists(data['train']))
print("Validation path:", os.path.exists(data['val']))
print("Test path:", os.path.exists(data['test']))

def count_images(directory):
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    return len([name for name in os.listdir(directory) if name.lower().endswith(supported_formats)])

print(f" The number of images in training set: {count_images(data['train'])}")
print(f" The number of images in validation set: {count_images(data['val'])}")
print(f" The number of images in test set: {count_images(data['test'])}")


Train path: True
Validation path: True
Test path: True
 The number of images in training set: 537
 The number of images in validation set: 41
 The number of images in test set: 19


In [2]:
from ultralytics import YOLO
from pathlib import Path

absolute_from_relative_path = Path('data.yaml').resolve()

model = YOLO('yolov8n-pose.pt')  # Load a pretrained model (recommended for training)
results = model.train(data=absolute_from_relative_path, epochs=60, imgsz=480)


Ultralytics YOLOv8.2.58 🚀 Python-3.9.18 torch-1.13.1 CPU (Apple M1)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=/Users/alessiacolumban/Desktop/Just_Pose/POSE.v3i.yolov8/data.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
train: Scanning /Users/alessiacolumban/Desktop/Just_Pose/POSE.v3i.yolov8/train/labels.cache... 537 images, 0 backgrounds, 0 corrupt: 100%|██████████| 537/537 [00:00<?, ?it/s]
val: Scanning /Users/alessiacolumban/Desktop/Just_Pose/POSE.v3i.yolov8/valid/labels.cache... 41 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<?, ?it/s]


Plotting labels to runs/pose/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 480 train, 480 val
Using 0 dataloader workers
Logging results to runs/pose/train2
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/34 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Save the trained model
results.save('my_trained_model.pt')

In [ ]:
# Load the trained model
model = YOLO('my_trained_model.pt')

In [ ]:
import cv2
from ultralytics import YOLO

# Load model
model = YOLO('my_trained_model.pt')  # Use your trained model

video_path = 0
cap = cv2.VideoCapture(video_path)

# Loop over the video frames
while cap.isOpened():
    # Read frame 
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame, save=True)

        # Visualize the results
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow('YOLOv8 Inference', annotated_frame)

        # Break the loop if "q" is pressed 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print('Video has ended or failed, exiting the loop')

# Release the video capture object and close the windows
cap.release()
cv2.destroyAllWindows()